# Run Hugging Face KoGPT-2 autoregressive sampling on Inf2 & Trn1

- References: https://github.com/aws-neuron/aws-neuron-samples/tree/master/torch-neuronx/transformers-neuronx


## Install Dependencies

필요 패키지

 - `torch-neuronx`
 - `neuronx-cc`
 - `transformers`
 - `transformers-neuronx`

참조: [torch-neuronx inference setup guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/frameworks/torch/torch-neuronx/setup/setup-inference.html)

In [ ]:
# !pip install git+https://github.com/aws-neuron/transformers-neuronx.git transformers -U

<br>

## 1. Compilation
---

### Load HF pre-trained model

In [1]:
import time
import torch
from transformers_neuronx.module import save_pretrained_split
from transformers.models.gpt2 import GPT2LMHeadModel
from transformers_neuronx.gpt2.model import GPT2ForSampling

hf_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', low_cpu_mem_usage=True, torchscript=True)


### Split the model state_dict into multiple files

호스트 메모리 사용량을 줄이기 위해, torch에서 기본적으로 제공하는 state_dict 메서드 대신 `transformers_neuronx` 에서 제공하는 `save_pretrained_split`를 사용할 수 있습니다. 
그리고 컴파일 및 배포 중에 메모리 사용량을 줄이기 위해 어텐션과 MLP 레이어를 FP16으로 캐스팅합니다. 이는 모델에서 각 레이어를 캐스팅하는 콜백 함수로 구현하면 됩니다.

In [2]:
def amp_callback(model, dtype):
    # cast attention and mlp to low precisions only; layernorms stay as f32
    for block in model.transformer.h:
        block.attn.to(dtype)
        block.mlp.to(dtype)
    model.lm_head.to(dtype)

amp_callback(hf_model, torch.float16)
save_pretrained_split(hf_model, './gpt2-split')

### Perform autoregressive sampling using tensor parallelism

대규모 언어 모델을 Inf2 및 Trn1에서 작동시키기 위해 텐서 병렬화를 적용하여 모델 파라미터를 여러 뉴런코어-v2로 분할합니다. 단일 뉴런코어에 할당될 메모리 양은 `tp_degree`로 설정하며 이 값이 높아질수록 추론 속도가 더 빨라지지만, 요구하는 메모리가 증가하기 때문에
인스턴스 사양과 모델 크기에 따라 적절한 크기를 설정해야 합니다.

`transformers_neuronx` 는 현 시점에서 동적 배칭을 지원하지 않기에, 모델 컴파일 시 `batch_size` 로 지정한 배치 개수만큼 추론해야 합니다.

In [3]:
# load GPT-2 to NeuronCores with 2-way tensor parallel
# enable float16 casting
neuron_model = GPT2ForSampling.from_pretrained('./gpt2-split', batch_size=4, tp_degree=2, amp='f16')
neuron_model.to_neuron()

..
Compiler status PASS
2023-Feb-20 08:30:57.0330 2184:4070 [1] ofi_init:1453 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2023-Feb-20 08:30:57.0330 2184:4070 [1] init.cc:101 CCOM WARN OFI plugin initNet() failed is EFA enabled?
..
Compiler status PASS
..
Compiler status PASS
..
Compiler status PASS


<br>

## 2. Inference
---


### Load tokenizer

In [4]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>', 
    padding_side='left'
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [7]:
def batch_generate_texts(batch_prompts, tokenizer, neuron_model, sequence_length=256):
    encodings = tokenizer.batch_encode_plus(batch_prompts, padding='longest', pad_to_max_length=True, return_tensors='pt')
    batch_input_ids, batch_attention_masks = encodings["input_ids"], encodings["attention_mask"]

    with torch.inference_mode():
        start = time.time()
        generated_ids = neuron_model.sample(batch_input_ids, sequence_length=sequence_length)
        elapsed = time.time() - start
        
    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return generated_texts

### Inference

In [9]:
%%time
batch_prompts = [
    "근육이 커지기 위해서는",
    "내일 날씨는",
    "수학은",
    "다윈의 진화론은"
]

batch_generate_texts(batch_prompts, tokenizer, neuron_model)

CPU times: user 1min 7s, sys: 80.2 ms, total: 1min 7s
Wall time: 1.08 s


['근육이 커지기 위해서는 수분섭취도 병행해야 한다.\n특히 입술을 자주 씻어주게 되면 입술 피부가 딱딱해질 수 있기 때문에 평소 뽀얀 피부를 유지하기 위한 노력을 병행한다.\n수분 섭취가 부족한 상태에서 물 없이 입안에 흡수된 물을 마시는 것은 피부가 건조해지는 것은 물론 자칫 얼굴로 돌아갈 수도 있기 때문에 자제해야 한다.\n또한 충분한 수분공급을 위해서는 각질제거를 위해 바르는 에센스와 크림 등과 같이 피부에 직접 닿아도 자극이 적은 천연 화장품을 선택하는 것이 좋다.\n특히 스킨, 로션 같은 보습제품의 사용은 피부를 매끄럽고 탄력 있게 가꾸는 데 매우 효과적이다.\n하지만 너무 많이 바를 경우 피부에 부담이 되므로 수분공급제 등으로 케어하는 것을 소홀히 해서는 안 된다.\n입술 피부의 건조함과 탄력을 높이는 것은 물론 보습효과가 뛰어난 크림으로 수분보충에 신경을 쓰는 것도 방법이다. 코로나 19 여파로 지난 3월 마스크 가격이 폭락하면서 방역당국이 사재기 행위를 중단하라는 국민 성원에 힘입어 소비진작에 나서면서 마스크 매출이 급증하고 있다.\n그러나 전문가들은 마스크를 구하기 어려운 소비자들도 여전히 마스크를 구입하는 경우가 많다고 지적한다.\n특히 마스크 가격의 급격한 하락으로 생필품을 구매하려는 이들이 많기 때문에 가격대가 저렴한 마스크를 구매해야 한다고 조언한다.\n한국은행은 1월 마',
 '내일 날씨는 예년과 비슷하게 구름 많다가 제주도는 밤부터 장맛비가 내리기 시작하겟다.\n내일까지 예상강수량은 제주도와 남부 내륙 5mm 안팎, 남해안, 내륙 5mm 안팎이다.\n제주도와 남해안 최고 150mm 이상, 제주도 산지 120mm 이상 폭우가 예상된다.\n중부지방과 경북 내륙에도 5~10mm의 비가 내릴 것으로 전망된다.\n기상청은 이번 장맛비의 예상 강수량은 제주도 50~100mm, 전남 남해안, 지리산 부근 80mm 내외다.\n예상 강수량은 제주도 30~60mm, 남해안과 울릉도·독도 10~30mm이고 강원 영동은 5mm 내외다.\n강원 산지에는 